In [1]:
import pandas as pd
import numpy as np
import datetime
import xlrd
%config Completer.use_jedi = False

In [2]:
stk_off = pd.DataFrame({'Ticker': ['EWZ US 05/29/20 P27', 'HSI US 12/30/20 C24800', 'TLT US 06/19/20 C225', 'EWZ US 05/29/20 P29', 'XLE US 05/29/20 P30'],
                   'Instrument': ['EWZ', 'HSI', 'TLT', 'EWZ', 'EWZ'],
                   'Maturity Label': ['MAI 20 D29', 'DEC 20 D30', 'JUN 20 D19', 'MAY 20 D29', 'MAY 20 D29'],
                   'Strike': [27, 24800, 225, 29, 29],
                   'Payout': ['P', 'C', 'C', 'P', 'P'],
                   'Style': ['A', 'E', 'E', 'A', 'A'],
                   'Maturity': ['29/05/2020', '12/30/2020', '19/06/2020', '29/05/2020', '29/05/2020'],
                   'Market': ['US NYSE', 'US NYSE', 'HK HKSE', 'US NYSE', 'US NYSE']})

stk_off_original = stk_off.copy()

stk_off['Reduced_Ticker'] = stk_off['Ticker'].apply(lambda a :" ".join(a.split(" ", 2)[:1]))

In [4]:
stk_off_original.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Ticker_Off.xlsx', index=False)

In [5]:
def organize_df(df):
    df = df[['Ticker', 'Instrument', 'Maturity Label', 'Strike', 'Payout', 'Style', 'Maturity', 'Market']]
    return df

In [6]:
def market_ticker(stk_off):
    market_ticker_var = stk_off[['Market','Reduced_Ticker']].groupby(['Market']).agg(list)
    market_ticker_var['Reduced_Ticker'] = market_ticker_var['Reduced_Ticker'].apply(lambda x: pd.unique(x))
    market_ticker_var = market_ticker_var.explode('Reduced_Ticker')
    market_ticker_var = market_ticker_var.reset_index()
    
    market_ticker_var.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Market_Ticker.xlsx', index=False)

In [7]:
market_ticker(stk_off)

In [8]:
def market_column(market_ticker, tickers, df):
    tickers_reduced = [t.split()[0] for t in tickers]
    
    selection_market = (market_ticker
                   .explode('Reduced_Ticker')
                   ['Reduced_Ticker'].apply(lambda x: x.split()[0] in tickers_reduced))


    market = market_ticker.loc[selection_market].Market.to_list()
        
    df['Market'] = market
    
    return df

In [9]:
def instrument_ticker(stk_off):
    instrument_ticker_var = stk_off[['Instrument','Reduced_Ticker']].groupby(['Instrument']).agg(list)
    instrument_ticker_var['Reduced_Ticker'] = instrument_ticker_var['Reduced_Ticker'].apply(lambda x: pd.unique(x))
    instrument_ticker_var = instrument_ticker_var.explode('Reduced_Ticker')
    instrument_ticker_var = instrument_ticker_var.reset_index()
    
    instrument_ticker_var.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Instrument_Ticker.xlsx', index=False)

In [10]:
instrument_ticker(stk_off)

In [11]:
def instrument_column(instrument_ticker, tickers, df):
    tickers_reduced = [t.split()[0] for t in tickers]
        
    selection_instrument = (instrument_ticker
             .explode('Reduced_Ticker')
             ['Reduced_Ticker'].apply(lambda x: x.split()[0] in tickers_reduced))

    instrument = instrument_ticker.loc[selection_instrument].Instrument.to_list()
     
    df['Instrument'] = instrument

    return df

In [12]:
def style(df):
    df['Style'] = ''
    for x in df['Market']:
        if x == 'US NYSE':
            df['Style'] = 'A'
        else:
            df['Style'] = 'E'
    
    return df

In [13]:
def payout_strike(df, tickers):
    payout_strike = [t.split()[3] for t in tickers]
    df['Payout'] = [p[0] for p in payout_strike]
    df['Strike'] = [s[1:] for s in payout_strike]
    
    return df

In [14]:
def maturity(df):
    df['Maturity'] = [datetime.datetime.strptime(x.split()[2], '%m/%d/%y').strftime('%d/%m/%Y') for x in df['Ticker']]
    return df

In [23]:
def read_maturities(market):
    
    if market == 'US NYSE':        
        maturities = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Vencimentos/US NYSE.xlsx')
        maturities['Maturity'] = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y') 
                     for x in maturities.Maturity]
        return maturities
    else:
        maturities = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Vencimentos/HK HKSE.xlsx')
        maturities['Maturity'] = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y') 
                     for x in maturities.Maturity]
        return maturities
        

In [24]:
x = read_maturities('US NYSE')
x

,Label,First Td,Maturity
0,MAI 20 D29,NaN,29/05/2020
1,DEC 20 D30,NaN,30/12/2020


In [30]:
def maturity_label(df, tickers):    
    date = [t.split()[2] for t in tickers]
    
    for m in df['Market']:
        maturities = read_maturities(m)
        if maturities['Maturity'].str.contains(m).any():
            selection_label = (maturities.explode('Maturity')['Maturity'].apply(lambda x: x in df['Maturity'].to_list()))                                                                                           
            df['Maturity Label'] = maturities_usnyse.loc[selection_label].Label.to_list()
        else:
            maturity_label = input('Data de Vencimento não cadastrada.\nInforme o Maturity Label:')
            df['Maturity Label'] = maturity_label
    return df

In [35]:
def file_to_upload(tickers):
    
    stk_off = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Ticker_Off.xlsx')
    stk_off_original = stk_off.copy()
    stk_off['Reduced_Ticker'] = stk_off['Ticker'].apply(lambda a :" ".join(a.split(" ", 2)[:2]))    
    tickers_to_upload = []
    msg = []
    
    for t in tickers:
        if stk_off_original['Ticker'].str.contains(t).any():
            msg.append('Ticker ' + t + ' já consta na base') 
        else:
            tickers_to_upload.append(t)         
            
    
    if not tickers_to_upload:
        msg.append("Os Tickers abaixo já constam na base")
        return tickers, msg[-1]
    
    df_to_upload = pd.DataFrame()
    df_to_upload['Ticker'] = tickers_to_upload

    market_ticker_var = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Market_Ticker.xlsx')
    df_to_upload = market_column(market_ticker_var, tickers_to_upload, df_to_upload)

    instrument_ticker_var = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Instrument_Ticker.xlsx')
    df_to_upload = instrument_column(instrument_ticker_var, tickers_to_upload, df_to_upload)

    df_to_upload = style(df_to_upload)

    df_to_upload = payout_strike(df_to_upload, tickers_to_upload)

    df_to_upload = maturity(df_to_upload)
    df_to_upload = maturity_label(df_to_upload,tickers)
    df_to_upload = organize_df(df_to_upload)
    
    stk_off_original = stk_off_original.append(df_to_upload, ignore_index=True)
    
    df_to_upload.to_excel('/home/gustavo/Desktop/Fut_Opt/Upload/Arquivo_para_upload.xlsx', index=False)
    stk_off_original.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Ticker_Off.xlsx', index=False)
    

    return df_to_upload, msg

In [36]:
def verify_ticker_by_Market(tickers):
    tic = [t.split()[0] for t in tickers]
    market_ticker = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Market_Ticker.xlsx')
    instrument_ticker = pd.read_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Instrument_Ticker.xlsx')
    for t in tic:
        if market_ticker['Reduced_Ticker'].str.contains(t).any():
            pass
        else:
            market = input("Não há nenhum mercado associado ao ticker " + t + ".\n Informe o Mercado:")
            to_add = [market, t]
            market_ticker.loc[len(market_ticker)] = to_add
            market_ticker.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Market_Ticker.xlsx', index=False)
        if instrument_ticker['Reduced_Ticker'].str.contains(t).any():
            pass
        else:
            instrument = input("Não há nenhum instrumento associado ao ticker " + t + ".\n Informe o Instrumento:")
            to_add = [instrument, t]
            instrument_ticker.loc[len(instrument_ticker)] = to_add
            instrument_ticker.to_excel('/home/gustavo/Desktop/Fut_Opt/Bases/Instrument_Ticker.xlsx', index=False)        
            
            

In [37]:
tickers = ['EWW US 05/29/20 P37', 'HSI US 12/30/21 C24880']
verify_ticker_by_Market(tickers)
x,y = file_to_upload(tickers)
print(y)
x

Data de Vencimento não cadastrada.
Informe o Maturity Label:DEC 21 D30
['Ticker EWW US 05/29/20 P37 já consta na base']


,Ticker,Instrument,Maturity Label,Strike,Payout,Style,Maturity,Market
0,HSI US 12/30/21 C24880,HSI,DEC 21 D30,24880,C,A,30/12/2021,US NYSE
